In [190]:
import math
import numpy as np
import pandas as pd

In [191]:
Elo_Score = pd.read_csv('fakeData.csv')

In [192]:
Elo_Score

,MSV,Elo,Home_Work_Score,Attendance_Score,Rate,Time
0,2020600201,1000,7,7,2,5
1,2020600202,1050,8,7,2,6
2,2020600203,580,9,7,2,4
3,2020600204,430,7,5,2,5
4,2020600205,2000,7,4,1,6
5,2020600206,787,8,7,1,3
6,2020600207,543,8,7,2,12
7,2020600208,2341,9,5,2,20
8,2020600209,3322,10,7,1,3
9,2020600210,655,9,7,0,24


In [193]:
class Calculate_Elo:
    def __init__(self, Dataframe_Elo_Score, TieuChiChinh, TieuChiPhu, K=32):
        self.Dataframe_Elo_Score = Dataframe_Elo_Score
        self.TieuChiChinh = TieuChiChinh
        self.TieuChiPhu = TieuChiPhu
        self.All_Score_Tieu_ChiChinh = None
        self.Dataframe_Elo_Score_sorted = None
        self.K = K


    def Calculate_Score_from_Tieu_Chi_chinh(self):
        self.Dataframe_Elo_Score["TieuChiChinh_Score"] = self.Dataframe_Elo_Score[self.TieuChiChinh].sum(axis=1)
        self.All_Score_Tieu_ChiChinh = self.Dataframe_Elo_Score["TieuChiChinh_Score"].values


    def score_d(self, elo_num, data):
        lst =  elo_num - data
        lst = [ 0.5 if item == 0  else  0 if item < 0 else 1 for item in lst]
        return np.array(lst).sum()


    def Probability(self, elo_num, data):
        list_pow = (elo_num - data) / 400.0
        list_pow = np.array([10**item for item in list_pow])
        return np.sum(1.0/(1+list_pow))


    def soc_iter(self, Elo_row, data, k = 12):
        d = self.score_d(Elo_row, data)
        Q = self.Probability(Elo_row, data)
        return Elo_row + k * (d-Q)
    

    def Calculate(self):
        self.Calculate_Score_from_Tieu_Chi_chinh()
        self.Dataframe_Elo_Score['Elo_After'] = self.Dataframe_Elo_Score.apply(lambda row: self.soc_iter(row['Elo'], self.All_Score_Tieu_ChiChinh, self.K), axis=1)
        
        # Sort data by Elo_After, then by TieuChiPhu
        self.Dataframe_Elo_Score_sorted = self.Dataframe_Elo_Score.sort_values(by=['Elo_After', *self.TieuChiPhu], ascending=False)
        return self.Dataframe_Elo_Score_sorted


In [194]:
cl = Calculate_Elo(Elo_Score.copy(), TieuChiChinh=['Home_Work_Score', 'Attendance_Score', 'Rate'], TieuChiPhu=['Time'], K=12)

In [195]:
cl.Calculate()

,MSV,Elo,Home_Work_Score,Attendance_Score,Rate,Time,TieuChiChinh_Score,Elo_After
8,2020600209,3322,10,7,1,3,18,3465.999999
7,2020600208,2341,9,5,2,20,16,2484.999780
4,2020600205,2000,7,4,1,6,12,2143.998432
1,2020600202,1050,8,7,2,6,17,1193.629156
0,2020600201,1000,7,7,2,5,16,1143.505896
5,2020600206,787,8,7,1,3,16,929.329896
10,2020600211,673,0,7,2,45,9,813.815106
9,2020600210,655,9,7,0,24,16,795.475908
2,2020600203,580,9,7,2,4,18,718.643951
6,2020600207,543,8,7,2,12,17,680.430599
